# OMIE Data Ingestion for Microsoft Fabric

This notebook downloads OMIE (Spanish electricity market) hourly price data and stores it in Microsoft Fabric lakehouse for bronze layer processing.

## Lakehouse Structure
- **Lakehouse**: `lkh_OMIE`
- **Path**: `Files/bronze/OMIE/`
- **Year-based organization**: `Files/bronze/OMIE/{year}/` (2023, 2024, 2025)
- **Manifest**: `Files/bronze/OMIE/omie_ingestion_manifest.parquet`

## Data Source
- **Source**: OMIE (Operador del Mercado Ibérico de Energía)
- **URL**: https://www.omie.es/es/file-access-list
- **Content**: Hourly electricity prices for Spanish daily market ("Precios horarios del mercado diario en España")

## Notes
- Files are automatically converted to Parquet format for optimal Fabric performance
- Direct download-to-Parquet conversion (no intermediate files)
- Uses PySpark when available (Fabric environment) with pandas fallback
- Year-based organization following Medallion architecture Bronze layer principles

In [28]:
import sys
import subprocess
import importlib

reqs = ['requests', 'beautifulsoup4', 'pandas', 'tqdm', 'openpyxl', 'pyarrow']
missing = [p for p in reqs if importlib.util.find_spec(p) is None]
if missing:
    print('Installing missing packages:', missing)
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', *missing])
    print('Installed packages; if imports still fail, please restart the kernel and re-run the notebook cells.')
else:
    print('All required packages are already installed')


Installing missing packages: ['beautifulsoup4']
Installed packages; if imports still fail, please restart the kernel and re-run the notebook cells.
Installed packages; if imports still fail, please restart the kernel and re-run the notebook cells.


In [ ]:
import re
import os
import time
from pathlib import Path
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, unquote
from tqdm.auto import tqdm
import pandas as pd

# Initialize Spark with Delta Lake support for Fabric
print("🔥 Initializing Spark session...")

try:
    from pyspark.sql import SparkSession
    
    # In Fabric, try to get existing session first
    try:
        spark = SparkSession.getActiveSession()
        if spark is None:
            # Create new session with Delta configuration
            spark = SparkSession.builder \
                .appName("OMIE_Data_Ingestion") \
                .getOrCreate()
        
        print(f'✅ Spark session active: {spark.version}')
        print(f'   📊 Spark UI: {spark.sparkContext.uiWebUrl}')
        
        # Test Delta Lake availability but disable due to OneLake issues
        try:
            from delta.tables import DeltaTable
            print('✅ Delta Lake libraries detected')
            print('⚠️  Delta Lake disabled due to OneLake connectivity issues')
            print('   💡 Using Parquet format for reliable operation')
            delta_available = False  # Force disable due to OneLake issues
        except ImportError as e:
            print(f'⚠️  Delta Lake not available: {e}')
            print('   💡 Using Parquet format')
            delta_available = False
            
    except Exception as e:
        print(f'⚠️  Error with existing session: {e}')
        # Try creating fresh session
        spark = SparkSession.builder \
            .appName("OMIE_Data_Ingestion") \
            .getOrCreate()
        print(f'✅ Created new Spark session: {spark.version}')
        delta_available = False

except Exception as e:
    spark = None
    delta_available = False
    print(f'❌ PySpark not available: {e}')
    print('   💡 Running in pandas-only mode')

# Force disable Delta Lake due to OneLake issues in Fabric
delta_available = False
print('\n🔺 Delta Lake Status: DISABLED')
print('   💡 Reason: OneLake connectivity issues in current Fabric environment')
print('   📦 Using Parquet format for reliable data storage')

# Global flag for Delta availability
globals()['DELTA_AVAILABLE'] = delta_available

# Settings
BASE_URL = 'https://www.omie.es'
TARGET_DIR = '/es/file-access-list?parents=/Mercado%20Diario/1.%20Precios&dir=Precios%20horarios%20del%20mercado%20diario%20en%20Espa%C3%B1a&realdir=marginalpdbc'
LIST_URL = BASE_URL + TARGET_DIR

# Fabric Lakehouse paths - ensure proper folder creation in Fabric environment
# Following the same pattern as OPEN-METEO: /lakehouse/default/Files/bronze/OMIE/
print("\n🏗️  Setting up OMIE lakehouse directory structure...")

# Always use Fabric lakehouse structure for consistency
if os.path.exists('/lakehouse/default/Files'):
    # Running in Microsoft Fabric - use default lakehouse structure
    LAKEHOUSE_ROOT = Path('/lakehouse/default/Files')
    print(f'✅ Detected Fabric environment - using lakehouse: {LAKEHOUSE_ROOT}')
else:
    # Local development - simulate Fabric structure
    LAKEHOUSE_ROOT = Path('lakehouse/default/Files')
    LAKEHOUSE_ROOT.mkdir(parents=True, exist_ok=True)
    print(f'💻 Local development mode - simulating Fabric structure: {LAKEHOUSE_ROOT}')

# Create the required Fabric folder structure: /lakehouse/default/Files/bronze/OMIE/
BRONZE_BASE_DIR = LAKEHOUSE_ROOT / 'bronze' / 'OMIE'
BRONZE_DIR = BRONZE_BASE_DIR  # Main bronze directory for OMIE
DOWNLOAD_DIR = BRONZE_BASE_DIR / 'downloads'  # Temporary downloads

# Ensure all directories exist
print(f"📁 Creating directory structure:")
print(f"   Bronze base: {BRONZE_BASE_DIR}")
print(f"   Bronze main: {BRONZE_DIR}")
print(f"   Downloads: {DOWNLOAD_DIR}")

# Create directories with proper permissions
BRONZE_BASE_DIR.mkdir(parents=True, exist_ok=True)
BRONZE_DIR.mkdir(parents=True, exist_ok=True)
DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)

HEADERS = {'User-Agent': 'omie-downloader/1.0 (+https://github.com)'}

print(f'\n📊 Environment Summary:')
print(f'   Spark available: {"✅ Yes" if spark else "❌ No"}')
print(f'   Delta available: {"✅ Yes" if DELTA_AVAILABLE else "❌ No (Disabled)"}')
print(f'   Format: 📦 Parquet (Reliable for Fabric)')
print(f'   Download directory: {DOWNLOAD_DIR}')
print(f'   Bronze directory: {BRONZE_DIR}')

pyspark not available in this environment: No module named 'pyspark'
🏗️  Setting up OMIE lakehouse directory structure...
💻 Local development mode - simulating Fabric structure: lakehouse\default\Files
📁 Creating directory structure:
   Bronze base: lakehouse\default\Files\bronze\OMIE
   Bronze main: lakehouse\default\Files\bronze\OMIE
   Downloads: lakehouse\default\Files\bronze\OMIE\downloads
Download directory: lakehouse\default\Files\bronze\OMIE\downloads
Bronze directory: lakehouse\default\Files\bronze\OMIE


In [30]:
# Clean up previous downloads and reset directories
import shutil
from pathlib import Path
import os

def clean_directories():
    """Clean and prepare directories for fresh data processing"""
    print("🧹 Cleaning directories for fresh processing...")
    
    # Remove downloads folder if it exists
    if DOWNLOAD_DIR.exists():
        shutil.rmtree(DOWNLOAD_DIR)
        print(f"   ✅ Removed downloads directory: {DOWNLOAD_DIR}")
    
    # Recreate downloads folder
    DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)
    print(f"   ✅ Created clean downloads directory: {DOWNLOAD_DIR}")
    
    # Ensure main bronze directory exists 
    BRONZE_DIR.mkdir(parents=True, exist_ok=True)
    print(f"   ✅ Ensured bronze directory exists: {BRONZE_DIR}")
    
    print("🏁 Directory cleanup completed!")

# Initialize the directory structure
clean_directories()

# Reset any global variables from previous runs
cleanup_vars = ['processed_files', 'downloaded', 'files', 'soup', 'df', 'manifest_df']
for var_name in cleanup_vars:
    if var_name in globals():
        del globals()[var_name]
        print(f"   🗑️ Cleared variable: {var_name}")

print("🧹 Cleanup completed! Ready for fresh download.")
print(f"📁 Download directory: {DOWNLOAD_DIR}")
print(f"📁 Bronze directory: {BRONZE_DIR}")

🧹 Cleaning directories for fresh processing...
   ✅ Removed downloads directory: lakehouse\default\Files\bronze\OMIE\downloads
   ✅ Created clean downloads directory: lakehouse\default\Files\bronze\OMIE\downloads
   ✅ Ensured bronze directory exists: lakehouse\default\Files\bronze\OMIE
🏁 Directory cleanup completed!
   🗑️ Cleared variable: processed_files
   🗑️ Cleared variable: files
   🗑️ Cleared variable: soup
   🗑️ Cleared variable: df
   🗑️ Cleared variable: manifest_df
🧹 Cleanup completed! Ready for fresh download.
📁 Download directory: lakehouse\default\Files\bronze\OMIE\downloads
📁 Bronze directory: lakehouse\default\Files\bronze\OMIE


In [31]:
# Import required libraries for this cell
import re
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, unquote

# Define constants at the top of the cell to ensure availability
BASE_URL = 'https://www.omie.es'
TARGET_DIR = '/es/file-access-list?parents=/Mercado%20Diario/1.%20Precios&dir=Precios%20horarios%20del%20mercado%20diario%20en%20Espa%C3%B1a&realdir=marginalpdbc'
LIST_URL = BASE_URL + TARGET_DIR
HEADERS = {'User-Agent': 'omie-downloader/1.0 (+https://github.com)'}

def inspect_omie_page(url=None, session=None, timeout=20):
    """Debug function to see what the OMIE page structure looks like"""
    # Use default URL if not provided
    if url is None:
        url = LIST_URL
    
    s = session or requests.Session()
    r = s.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, 'html.parser')
    
    print(f"Page title: {soup.title.string if soup.title else 'No title'}")
    print(f"Total links found: {len(soup.find_all('a'))}")
    
    # Look for table rows which typically contain file listings
    tables = soup.find_all('table')
    print(f"Tables found: {len(tables)}")
    
    # Look for file listing patterns in OMIE
    file_rows = []
    for table in tables:
        rows = table.find_all('tr')
        for row in rows:
            cells = row.find_all(['td', 'th'])
            if len(cells) >= 2:  # File listings usually have multiple columns
                row_text = ' '.join([cell.get_text(strip=True) for cell in cells])
                links = row.find_all('a')
                if links and any(keyword in row_text.lower() for keyword in ['csv', 'descarg', 'download', 'marginal']):
                    file_rows.append({
                        'row_text': row_text[:100],
                        'links': [{'href': a.get('href'), 'text': a.get_text(strip=True)} for a in links]
                    })
    
    print(f"\n🔍 Potential file rows found: {len(file_rows)}")
    for i, row in enumerate(file_rows[:5]):
        print(f"  Row {i+1}: {row['row_text']}")
        for link in row['links']:
            print(f"    -> {link['text']} : {link['href'][:60]}...")
    
    return soup

def fetch_file_list(url=None, session=None, timeout=20):
    """
    OMIE file list fetcher that looks for table-based file listings
    which is more common in OMIE's file access structure
    """
    # Use default URL if not provided
    if url is None:
        url = LIST_URL
        
    s = session or requests.Session()
    r = s.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, 'html.parser')
    
    files = []
    
    # Strategy 1: Look for tables containing file listings
    tables = soup.find_all('table')
    for table in tables:
        rows = table.find_all('tr')
        for row in rows:
            cells = row.find_all(['td', 'th'])
            links = row.find_all('a')
            
            if not links:
                continue
                
            # Check if this row contains file information
            row_text = ' '.join([cell.get_text(strip=True) for cell in cells])
            
            # Look for indicators this is a file row
            is_file_row = any([
                # Date patterns (OMIE files often have dates)
                re.search(r'\d{2}[-/]\d{2}[-/]\d{4}', row_text),
                re.search(r'\d{4}[-/]\d{2}[-/]\d{2}', row_text),
                # File size patterns
                re.search(r'\d+\s*(kb|mb|bytes)', row_text, re.IGNORECASE),
                # Download/file keywords
                any(kw in row_text.lower() for kw in ['descarg', 'download', 'csv', 'marginal']),
                # Time patterns (for hourly data)
                re.search(r'\d{2}:\d{2}', row_text)
            ])
            
            if is_file_row:
                for link in links:
                    href = link.get('href')
                    if href and href != '#':
                        # Get filename from link text or URL
                        link_text = link.get_text(strip=True)
                        
                        # Prefer meaningful link text over URL-derived names
                        if link_text and len(link_text) > 3 and len(link_text) < 100:
                            name = link_text
                        else:
                            # Extract from URL
                            parsed = urlparse(href)
                            name = unquote(parsed.path.split('/')[-1])
                            if not name or name == '' or '.' not in name:
                                # Use query parameters if available
                                from urllib.parse import parse_qs
                                query_params = parse_qs(parsed.query)
                                if 'file' in query_params:
                                    name = query_params['file'][0]
                                elif any(param in query_params for param in ['id', 'document', 'doc']):
                                    # Generic name with ID
                                    param_val = next((v[0] for k, v in query_params.items() if k in ['id', 'document', 'doc']), '')
                                    name = f"omie_document_{param_val}"
                                else:
                                    name = f"omie_file_{len(files)}"
                        
                        full_url = urljoin(BASE_URL, href)
                        
                        files.append({
                            'name': name,
                            'href': full_url,
                            'link_text': link_text,
                            'row_context': row_text[:200]  # Keep context for debugging
                        })
    
    # Strategy 2: Fallback to looking for direct download links
    if not files:
        print("🔄 No table-based files found, trying direct link approach...")
        for a in soup.find_all('a'):
            href = a.get('href')
            if not href:
                continue
            
            # Look for obvious download patterns
            if any([
                'download' in href.lower(),
                'descarg' in href.lower(),
                re.search(r'\.(csv|xls|xlsx|zip|txt)(\?|$)', href, re.IGNORECASE),
                'marginalpdbc' in href.lower()
            ]):
                link_text = a.get_text(strip=True)
                name = link_text if link_text and len(link_text) < 100 else f"omie_download_{len(files)}"
                full_url = urljoin(BASE_URL, href)
                
                files.append({
                    'name': name,
                    'href': full_url,
                    'link_text': link_text,
                    'row_context': ''
                })
    
    # Remove duplicates based on URL
    seen_urls = set()
    unique_files = []
    
    for file_info in files:
        url = file_info['href']
        if url not in seen_urls:
            seen_urls.add(url)
            unique_files.append(file_info)
    
    return unique_files

# Update global variables for the rest of the notebook
globals()['BASE_URL'] = BASE_URL
globals()['TARGET_DIR'] = TARGET_DIR
globals()['LIST_URL'] = LIST_URL
globals()['HEADERS'] = HEADERS

print('✅ Prepared improved file list functions with OMIE-specific parsing')
print(f'🌐 Using URL: {LIST_URL}')

✅ Prepared improved file list functions with OMIE-specific parsing
🌐 Using URL: https://www.omie.es/es/file-access-list?parents=/Mercado%20Diario/1.%20Precios&dir=Precios%20horarios%20del%20mercado%20diario%20en%20Espa%C3%B1a&realdir=marginalpdbc


In [32]:
# Import required libraries for this cell
import re
import pandas as pd

# Test the improved OMIE file fetching
print("🔍 Testing improved OMIE file detection...")

# First inspect the page structure
print("=" * 60)
print("🔍 INSPECTING OMIE PAGE STRUCTURE")
print("=" * 60)
soup = inspect_omie_page()

print("\n" + "=" * 60)
print("🔍 TESTING IMPROVED FILE LIST FUNCTION") 
print("=" * 60)

# Test the improved file list function
files = fetch_file_list()
print(f'✅ Found {len(files)} unique file candidates')

if files:
    # Create DataFrame for analysis
    df = pd.DataFrame(files)
    
    print(f"\n📋 First 10 files found:")
    display(df[['name', 'href']].head(10))
    
    # Quality checks
    print(f"\n🔍 QUALITY ANALYSIS:")
    
    # Check URL patterns
    url_patterns = {}
    for url in df['href']:
        if 'download' in url.lower():
            pattern = 'download_action'
        elif any(ext in url.lower() for ext in ['.csv', '.xlsx', '.xls']):
            pattern = 'direct_file'  
        elif 'marginalpdbc' in url.lower():
            pattern = 'marginal_data'
        elif re.search(r'[?&](file|id|doc)=', url):
            pattern = 'parameterized'
        else:
            pattern = 'other'
        url_patterns[pattern] = url_patterns.get(pattern, 0) + 1
    
    print("📊 URL Pattern Analysis:")
    for pattern, count in url_patterns.items():
        print(f"   {pattern}: {count} files")
    
    # Check for potential duplicates
    duplicate_names = df['name'].value_counts()
    duplicates = duplicate_names[duplicate_names > 1]
    
    if len(duplicates) > 0:
        print(f"\n⚠️  DUPLICATE NAMES FOUND ({len(duplicates)} names):")
        for name, count in duplicates.head(5).items():
            print(f"   '{name}' appears {count} times")
            # Show the URLs for these duplicates
            duplicate_urls = df[df['name'] == name]['href'].tolist()
            for i, url in enumerate(duplicate_urls[:3]):
                print(f"     {i+1}. {url[:80]}...")
    else:
        print("✅ No duplicate names found")
    
    # Check for duplicate URLs (the main problem we're trying to solve)
    duplicate_urls = df['href'].value_counts() 
    url_duplicates = duplicate_urls[duplicate_urls > 1]
    
    if len(url_duplicates) > 0:
        print(f"\n❌ DUPLICATE URLS FOUND ({len(url_duplicates)} URLs):")
        print("   This is the main issue causing identical downloads!")
        for url, count in url_duplicates.head(3).items():
            print(f"   URL appears {count} times: {url[:80]}...")
    else:
        print("✅ No duplicate URLs found - this should fix the identical downloads!")
    
    # Show some sample row contexts for debugging
    print(f"\n🔍 Sample file contexts (first 3):")
    for i, row in df.head(3).iterrows():
        print(f"   {i+1}. {row['name']}")
        print(f"      URL: {row['href'][:80]}...")
        if row.get('row_context'):
            print(f"      Context: {row['row_context'][:100]}...")
        print()

else:
    print("❌ No files found!")
    print("   The OMIE page structure might have changed.")
    print("   Check the page inspection output above for clues.")

🔍 Testing improved OMIE file detection...
🔍 INSPECTING OMIE PAGE STRUCTURE
Page title: Precios horarios del mercado diario en España | OMIE
Total links found: 1054
Tables found: 1
Page title: Precios horarios del mercado diario en España | OMIE
Total links found: 1054
Tables found: 1

🔍 Potential file rows found: 1010
  Row 1: marginalpdbc_20251001.1 2.62 KB 30/09/2025  14:13:55
    -> marginalpdbc_20251001.1 : /es/file-download?parents=marginalpdbc&filename=marginalpdbc...
  Row 2: marginalpdbc_20250930.1 633 bytes 29/09/2025  13:12:10
    -> marginalpdbc_20250930.1 : /es/file-download?parents=marginalpdbc&filename=marginalpdbc...
  Row 3: marginalpdbc_20250929.1 625 bytes 29/09/2025  05:06:17
    -> marginalpdbc_20250929.1 : /es/file-download?parents=marginalpdbc&filename=marginalpdbc...
  Row 4: marginalpdbc_20250928.1 659 bytes 28/09/2025  05:19:02
    -> marginalpdbc_20250928.1 : /es/file-download?parents=marginalpdbc&filename=marginalpdbc...
  Row 5: marginalpdbc_20250927.1 657 b

,name,href
0,marginalpdbc_20251001.1,https://www.omie.es/es/file-download?parents=m...
1,marginalpdbc_20250930.1,https://www.omie.es/es/file-download?parents=m...
2,marginalpdbc_20250929.1,https://www.omie.es/es/file-download?parents=m...
3,marginalpdbc_20250928.1,https://www.omie.es/es/file-download?parents=m...
4,marginalpdbc_20250927.1,https://www.omie.es/es/file-download?parents=m...
5,marginalpdbc_20250926.1,https://www.omie.es/es/file-download?parents=m...
6,marginalpdbc_20250925.1,https://www.omie.es/es/file-download?parents=m...
7,marginalpdbc_20250924.1,https://www.omie.es/es/file-download?parents=m...
8,marginalpdbc_20250923.1,https://www.omie.es/es/file-download?parents=m...
9,marginalpdbc_20250922.1,https://www.omie.es/es/file-download?parents=m...



🔍 QUALITY ANALYSIS:
📊 URL Pattern Analysis:
   download_action: 1010 files
✅ No duplicate names found
✅ No duplicate URLs found - this should fix the identical downloads!

🔍 Sample file contexts (first 3):
   1. marginalpdbc_20251001.1
      URL: https://www.omie.es/es/file-download?parents=marginalpdbc&filename=marginalpdbc_...
      Context: marginalpdbc_20251001.1 2.62 KB 30/09/2025  14:13:55...

   2. marginalpdbc_20250930.1
      URL: https://www.omie.es/es/file-download?parents=marginalpdbc&filename=marginalpdbc_...
      Context: marginalpdbc_20250930.1 633 bytes 29/09/2025  13:12:10...

   3. marginalpdbc_20250929.1
      URL: https://www.omie.es/es/file-download?parents=marginalpdbc&filename=marginalpdbc_...
      Context: marginalpdbc_20250929.1 625 bytes 29/09/2025  05:06:17...



In [33]:
# Download helper functions with optimized path handling
import time
from urllib.parse import urlparse

def download_file(file_meta, dest_dir=None, session=None, max_retries=3, chunk_size=1024*64):
    """Download a file with retry logic and streaming"""
    # Use global DOWNLOAD_DIR if not specified
    if dest_dir is None:
        dest_dir = DOWNLOAD_DIR
        
    s = session or requests.Session()
    url = file_meta['href']
    name = file_meta.get('name') or Path(urlparse(url).path).name
    dest = Path(dest_dir) / name
    
    if dest.exists():
        return dest
        
    for attempt in range(1, max_retries+1):
        try:
            with s.get(url, headers=HEADERS, stream=True, timeout=30) as r:
                r.raise_for_status()
                with open(dest, 'wb') as fh:
                    for chunk in r.iter_content(chunk_size=chunk_size):
                        if chunk:
                            fh.write(chunk)
            return dest
        except Exception as e:
            print(f'Attempt {attempt} failed: {e}')
            time.sleep(1.5 * attempt)
    
    raise RuntimeError(f'Failed to download {url} after {max_retries} attempts')

def load_into_spark(path, spark_session=None):
    """Load various file formats into Spark DataFrame"""
    # Try to get spark from globals or parameter
    spark = spark_session
    if spark is None and 'spark' in globals():
        spark = globals()['spark']
    
    if spark is None:
        raise RuntimeError('No SparkSession available')
    
    p = str(path)
    lower = p.lower()
    if lower.endswith('.csv'):
        return spark.read.option('header', 'true').option('inferSchema', 'true').csv(p)
    if lower.endswith(('.xls', '.xlsx')):
        import pandas as _pd
        df = _pd.read_excel(p)
        return spark.createDataFrame(df)
    if lower.endswith('.parquet'):
        return spark.read.parquet(p)
    return spark.read.option('header', 'true').option('inferSchema', 'true').csv(p)

print('✅ Download functions prepared')
print(f'📁 Download directory: {DOWNLOAD_DIR}')
print(f'📁 Bronze directory: {BRONZE_DIR}')

✅ Download functions prepared
📁 Download directory: lakehouse\default\Files\bronze\OMIE\downloads
📁 Bronze directory: lakehouse\default\Files\bronze\OMIE


In [ ]:
# Download and process files for Fabric Bronze Layer with year organization
import re
from datetime import datetime

print("🚀 Starting OMIE data ingestion for Fabric Bronze Layer...")
print(f"🔺 Delta Lake available: {'✅ Yes' if DELTA_AVAILABLE else '❌ No'}")
print(f"📊 Format target: {'Delta Tables' if DELTA_AVAILABLE else 'Parquet Files'}")

def extract_date_from_filename(filename):
    """Extract date from OMIE filename like 'marginalpdbc_20230101.1' -> (2023, '20230101')"""
    # Look for date patterns in the filename
    date_patterns = [
        r'(\d{8})',        # YYYYMMDD format like 20230101
        r'(\d{4}\d{2}\d{2})', # Alternative YYYYMMDD
    ]
    
    for pattern in date_patterns:
        match = re.search(pattern, filename)
        if match:
            date_str = match.group(1)
            if len(date_str) == 8:  # YYYYMMDD
                year = int(date_str[:4])
                return year, date_str
    
    return None, None

def filter_files_by_years(files, target_years=[2023, 2024, 2025]):
    """Filter files to only include those from target years"""
    filtered_files = []
    year_stats = {year: 0 for year in target_years}
    
    for file_info in files:
        filename = file_info.get('name', '')
        year, date_str = extract_date_from_filename(filename)
        
        if year and year in target_years:
            file_info['extracted_year'] = year
            file_info['extracted_date'] = date_str
            filtered_files.append(file_info)
            year_stats[year] += 1
    
    print(f"📊 Files found by year:")
    for year, count in year_stats.items():
        print(f"   {year}: {count} files")
    
    return filtered_files, year_stats

def create_year_directories(base_dir, years):
    """Create year-based directory structure directly under bronze/OMIE"""
    year_dirs = {}
    for year in years:
        year_dir = base_dir / str(year)
        year_dir.mkdir(parents=True, exist_ok=True)
        year_dirs[year] = year_dir
    return year_dirs

def save_as_parquet_file(df, file_path, file_name):
    """Save DataFrame as Parquet file"""
    try:
        df.to_parquet(file_path, engine='pyarrow', index=False)
        
        file_size = file_path.stat().st_size
        rows = len(df)
        
        print(f'     ✅ Parquet file saved: {file_name}')
        print(f'        📊 Size: {file_size/1024/1024:.2f} MB, Rows: {rows:,}')
        
        return True, file_size, rows, 'parquet'
        
    except Exception as e:
        print(f'     ❌ Parquet save failed: {e}')
        return False, 0, 0, 'error'

# Get file list (try resume files first, then fetch new)
if 'filtered_files_resume' in globals() and filtered_files_resume:
    print("🔄 Using filtered files from resume processing...")
    filtered_files = filtered_files_resume
    files = filtered_files  # For compatibility
    
    # Calculate year stats from resume files
    year_stats = {}
    for file_info in filtered_files:
        year = file_info['extracted_year']
        year_stats[year] = year_stats.get(year, 0) + 1
    
    print(f"📊 Resume files by year:")
    for year, count in year_stats.items():
        print(f"   {year}: {count} files")
        
else:
    # Fetch fresh file list
    try:
        files = fetch_file_list()
        print(f'✓ Found {len(files)} total files from OMIE')
    except Exception as e:
        print(f'❌ Failed to fetch file list: {e}')
        files = []

    if not files:
        print("No files to process. Exiting.")
        filtered_files = []
        year_stats = {}
    else:
        # Filter files for 2023-2025
        target_years = [2023, 2024, 2025]
        filtered_files, year_stats = filter_files_by_years(files, target_years)

if filtered_files:
    print(f"\n🎯 Processing {len(filtered_files)} files from {list(year_stats.keys())}")
    
    # Create year-based directory structure for Parquet files
    print(f"\n📁 Creating year-based directories...")
    
    target_years = [2023, 2024, 2025]
    bronze_year_dirs = create_year_directories(BRONZE_DIR, target_years)
    
    print(f"   Bronze directories created:")
    for year, path in bronze_year_dirs.items():
        print(f"     {year}: {path}")
    
    # Filter settings for OMIE price data
    FILTER_PATTERNS = ['marginalpdbc']  # Focus on price data files
    
    def matches_filters(name, patterns):
        if not patterns:
            return True
        lower = name.lower()
        return any(p.lower() in lower for p in patterns)
    
    def get_safe_name(original_name, year):
        """Convert original filename to safe name for Parquet file"""
        stem = Path(original_name).stem.replace('.', '_').replace('-', '_')
        return f"omie_{year}_{stem}"
    
    # Process files
    processed_files = []
    files_to_process = filtered_files
    
    print(f"\n📥 Processing {len(files_to_process)} files...")
    print(f"🎯 Target format: 📦 Parquet Files (Reliable)")
    
    # Group files by year for better organization
    files_by_year = {}
    for file_info in files_to_process:
        year = file_info['extracted_year']
        if year not in files_by_year:
            files_by_year[year] = []
        files_by_year[year].append(file_info)
    
    # Process files year by year
    for year in sorted(files_by_year.keys()):
        year_files = files_by_year[year]
        bronze_year_dir = bronze_year_dirs[year]
        
        print(f"\n🗓️ Processing {len(year_files)} files for {year} → {bronze_year_dir}")
        
        for i, meta in enumerate(year_files):
            original_name = meta.get('name') or f'file_{i}'
            year = meta['extracted_year']
            date_str = meta['extracted_date']
            
            print(f"   [{i+1}/{len(year_files)}] {original_name}")
            
            if not matches_filters(original_name, FILTER_PATTERNS):
                print(f'     ⏭️ Skipping (not price data): {original_name}')
                continue
            
            try:
                # Generate safe name for Parquet file
                safe_name = get_safe_name(original_name, year)
                target_path = bronze_year_dir / f"{safe_name}.parquet"
                
                # Skip if already exists
                if target_path.exists():
                    print(f'     ℹ️ File exists: {safe_name}.parquet')
                    # Get size for tracking
                    try:
                        file_size = target_path.stat().st_size
                        processed_files.append({
                            'original_name': original_name,
                            'source_url': meta.get('href'),
                            'bronze_path': str(target_path),
                            'file_size': file_size,
                            'year': year,
                            'date': date_str,
                            'processed_at': datetime.now().isoformat(),
                            'status': 'existing',
                            'format': 'parquet'
                        })
                    except:
                        pass
                    continue
                
                # Download and process file
                print(f'     📥 Downloading and converting to Parquet...')
                
                import requests
                import tempfile
                import pandas as pd
                
                # Download to temporary location
                response = requests.get(meta['href'], headers=HEADERS, timeout=30)
                response.raise_for_status()
                
                # Determine file type and process accordingly
                original_ext = Path(original_name).suffix.lower()
                
                with tempfile.NamedTemporaryFile(suffix=original_ext, delete=False) as temp_file:
                    temp_file.write(response.content)
                    temp_file_path = temp_file.name
                
                try:
                    # Convert to DataFrame based on file type
                    if original_ext == '.csv':
                        df = pd.read_csv(temp_file_path)
                    elif original_ext in ['.xls', '.xlsx']:
                        df = pd.read_excel(temp_file_path)
                    elif original_ext == '.txt':
                        # Try to read as CSV first, fallback to read as text
                        try:
                            df = pd.read_csv(temp_file_path, sep=None, engine='python')
                        except:
                            # If CSV parsing fails, read as raw text
                            with open(temp_file_path, 'r', encoding='utf-8') as f:
                                content = f.read()
                            df = pd.DataFrame({'raw_content': [content]})
                    else:
                        # For unknown formats, try CSV parsing or save as raw text
                        try:
                            df = pd.read_csv(temp_file_path, sep=None, engine='python')
                        except:
                            with open(temp_file_path, 'r', encoding='utf-8', errors='ignore') as f:
                                content = f.read()
                            df = pd.DataFrame({'raw_content': [content]})
                    
                    # Add metadata columns for better Bronze layer tracking
                    df['_source_file'] = original_name
                    df['_source_url'] = meta.get('href')
                    df['_extraction_year'] = year
                    df['_extraction_date'] = date_str
                    df['_ingested_at'] = datetime.now().isoformat()
                    
                    # Save as Parquet file
                    success, file_size, rows, format_used = save_as_parquet_file(
                        df, target_path, f"{safe_name}.parquet"
                    )
                    
                    if success:
                        processed_files.append({
                            'original_name': original_name,
                            'source_url': meta.get('href'),
                            'bronze_path': str(target_path),
                            'file_size': file_size,
                            'rows': rows,
                            'year': year,
                            'date': date_str,
                            'processed_at': datetime.now().isoformat(),
                            'status': 'processed',
                            'format': format_used
                        })
                    else:
                        print(f'     ❌ Failed to save file')
                    
                finally:
                    # Clean up temporary file
                    import os
                    try:
                        os.unlink(temp_file_path)
                    except:
                        pass
                
            except Exception as e:
                print(f'     ❌ Failed to process {original_name}: {e}')
                import traceback
                traceback.print_exc()
    
    print(f'\n🎉 Successfully processed {len(processed_files)} files to Fabric Bronze Layer')
    print(f'📁 Files organized by year in: {BRONZE_DIR}')
    
    # Show comprehensive summary by year
    if processed_files:
        summary_df = pd.DataFrame(processed_files)
        
        print(f"\n📋 Fabric Bronze Layer Summary by Year:")
        for year in sorted(summary_df['year'].unique()):
            year_data = summary_df[summary_df['year'] == year]
            total_size = year_data['file_size'].sum() / 1024 / 1024
            total_rows = year_data['rows'].sum() if 'rows' in year_data.columns else 0
            new_files = len(year_data[year_data['status'] == 'processed'])
            existing_files = len(year_data[year_data['status'] == 'existing'])
            
            print(f"   📅 {year}: {len(year_data)} files ({new_files} new, {existing_files} existing)")
            print(f"       💾 Size: {total_size:.2f} MB")
            if total_rows > 0:
                print(f"       📊 Rows: {total_rows:,}")
            print(f"       📂 Location: {BRONZE_DIR}/{year}/")
        
        print(f"\n📊 Overall Fabric Bronze Layer Statistics:")
        print(f"   📁 Total files: {len(summary_df)}")
        print(f"   💾 Total size: {summary_df['file_size'].sum() / 1024 / 1024:.2f} MB")
        print(f"   📅 Years covered: {sorted(summary_df['year'].unique())}")
        print(f"   📦 Format: Parquet (Reliable for Fabric)")
        print(f"   🏗️ Ready for Silver layer processing")
        
        # Show sample of processed files
        print(f"\n📋 Sample of processed files:")
        display(summary_df[['original_name', 'year', 'file_size', 'format', 'status']].head(10))
    else:
        print("❌ No files were processed successfully.")
        
    # Make processed_files globally available for manifest creation
    globals()['processed_files'] = processed_files
    globals()['year_stats'] = year_stats
    
    # Clear resume files if used
    if 'filtered_files_resume' in globals():
        del globals()['filtered_files_resume']
        print("🧹 Cleared resume files from memory")
    
    print(f"\n✅ Bronze layer ready with Parquet format!")
    print(f"📦 Parquet files created - reliable and ready for analytics")
    print(f"💡 Next step: Create Silver layer transformations")

else:
    print("❌ No files found to process.")

🚀 Starting OMIE data ingestion for Fabric Bronze Layer...
✓ Found 1010 total files from OMIE
📊 Files found by year:
   2023: 365 files
   2024: 366 files
   2025: 274 files

🎯 Filtered to 1005 files from [2023, 2024, 2025]

📁 Creating simplified year-based directories for Fabric...
   Bronze directories created:
     2023: lakehouse\default\Files\bronze\OMIE\2023
     2024: lakehouse\default\Files\bronze\OMIE\2024
     2025: lakehouse\default\Files\bronze\OMIE\2025

📥 Processing 1005 files for Fabric Bronze Layer...

🗓️ Processing 365 files for 2023 → lakehouse\default\Files\bronze\OMIE\2023
   [1/365] marginalpdbc_20231231.1
     📥 Downloading and converting to Parquet...
✓ Found 1010 total files from OMIE
📊 Files found by year:
   2023: 365 files
   2024: 366 files
   2025: 274 files

🎯 Filtered to 1005 files from [2023, 2024, 2025]

📁 Creating simplified year-based directories for Fabric...
   Bronze directories created:
     2023: lakehouse\default\Files\bronze\OMIE\2023
     2024:

,original_name,year,file_size,status
0,marginalpdbc_20231231.1,2023,5315,processed
1,marginalpdbc_20231230.1,2023,5314,processed
2,marginalpdbc_20231229.1,2023,5331,processed
3,marginalpdbc_20231228.1,2023,5339,processed
4,marginalpdbc_20231227.1,2023,5329,processed
5,marginalpdbc_20231226.1,2023,5326,processed
6,marginalpdbc_20231225.1,2023,5302,processed
7,marginalpdbc_20231224.1,2023,5309,processed
8,marginalpdbc_20231223.1,2023,5320,processed
9,marginalpdbc_20231222.1,2023,5305,processed



✅ Bronze layer ready for Fabric Medallion architecture!
💡 Next step: Create Silver layer transformations in Fabric


In [37]:
# Create manifest and verify lakehouse integration with Delta format support
print("📝 Creating ingestion manifest and verifying Fabric lakehouse structure...")

# First, verify our directory structure is visible in Fabric
print("🔍 Verifying Fabric lakehouse directory structure:")
print(f"   Lakehouse root: {LAKEHOUSE_ROOT}")
print(f"   Bronze base: {BRONZE_BASE_DIR}")  
print(f"   Bronze OMIE: {BRONZE_DIR}")
print(f"   Downloads: {DOWNLOAD_DIR}")

# List the actual directory contents to confirm they're visible
if BRONZE_BASE_DIR.exists():
    print(f"\n📂 Contents of {BRONZE_BASE_DIR}:")
    for item in BRONZE_BASE_DIR.iterdir():
        if item.is_dir():
            # Check if it's a Delta table (contains _delta_log)
            delta_log_path = item / '_delta_log'
            if delta_log_path.exists():
                item_count = len(list(item.glob('*.parquet')))
                print(f"   🔺 {item.name}/ (Delta table, {item_count} parquet files)")
            else:
                item_count = len(list(item.iterdir())) if item.exists() else 0
                print(f"   📁 {item.name}/ ({item_count} items)")
        else:
            size_mb = item.stat().st_size / 1024 / 1024
            print(f"   📄 {item.name} ({size_mb:.2f} MB)")
else:
    print(f"❌ Bronze directory does not exist: {BRONZE_BASE_DIR}")

# Check if we have processed files from this session
if 'processed_files' in globals() and processed_files:
    # Create manifest DataFrame
    manifest_df = pd.DataFrame(processed_files)
    
    # Save manifest to lakehouse as both Delta and CSV for compatibility
    try:
        # Try to save as Delta table for better integration
        if spark is not None:
            spark_manifest = spark.createDataFrame(manifest_df)
            manifest_delta_path = BRONZE_DIR / 'omie_ingestion_manifest_delta'
            spark_manifest.write.format("delta").mode("overwrite").save(str(manifest_delta_path))
            print(f'✅ Delta manifest saved: {manifest_delta_path}')
        else:
            print('⚠️  No Spark available, skipping Delta manifest')
    except Exception as e:
        print(f'⚠️  Could not save Delta manifest: {e}')
    
    # Also save as Parquet for backup
    manifest_parquet_path = BRONZE_DIR / 'omie_ingestion_manifest.parquet'
    manifest_df.to_parquet(str(manifest_parquet_path), engine='pyarrow', index=False)
    print(f'✅ Parquet manifest saved: {manifest_parquet_path}')
    
    # Also save as CSV for easy viewing
    csv_manifest_path = BRONZE_DIR / 'omie_ingestion_manifest.csv'
    manifest_df.to_csv(str(csv_manifest_path), index=False)
    print(f'✅ CSV manifest saved: {csv_manifest_path}')
    
    # Show format breakdown
    format_counts = manifest_df['format'].value_counts()
    print(f'\n📊 Ingestion Summary:')
    print(f'   Files processed this session: {len(manifest_df)}')
    print(f'   Total size: {manifest_df["file_size"].sum() / 1024 / 1024:.2f} MB')
    print(f'   Format breakdown:')
    for format_type, count in format_counts.items():
        print(f'     🔺 {format_type}: {count} files')
    print(f'   Lakehouse location: {BRONZE_DIR}')
    
else:
    print('ℹ️  No processed files found from this session.')
    print('   This could mean:')
    print('   1. The previous cell needs to be run first')
    print('   2. There was an error during processing')
    print('   3. Files were processed in a previous session')

print(f'\n📁 Current Lakehouse Structure:')
print(f'   Lakehouse root: {LAKEHOUSE_ROOT}')
print(f'   Bronze directory: {BRONZE_DIR}')
print(f'   Download directory: {DOWNLOAD_DIR}')

# List ALL files and Delta tables in bronze directory
if BRONZE_DIR.exists():
    # Walk through all subdirectories to find Delta tables and files
    all_items = []
    
    for year_dir in BRONZE_DIR.iterdir():
        if year_dir.is_dir() and year_dir.name.isdigit():  # Year directories
            print(f'\n📅 Year {year_dir.name}:')
            
            for item in year_dir.iterdir():
                if item.is_dir():
                    # Check if it's a Delta table
                    delta_log_path = item / '_delta_log'
                    if delta_log_path.exists():
                        # It's a Delta table
                        parquet_files = list(item.glob('*.parquet'))
                        table_size = sum(f.stat().st_size for f in parquet_files) / 1024 / 1024
                        print(f'   🔺 {item.name}/ (Delta table, {len(parquet_files)} files, {table_size:.2f} MB)')
                        
                        # Try to show Delta table info if possible
                        if spark is not None:
                            try:
                                from delta.tables import DeltaTable
                                if DeltaTable.isDeltaTable(spark, str(item)):
                                    delta_table = DeltaTable.forPath(spark, str(item))
                                    df_info = delta_table.toDF()
                                    row_count = df_info.count()
                                    print(f'       📊 Rows: {row_count:,}')
                            except Exception as e:
                                print(f'       ⚠️  Could not read Delta table: {e}')
                    else:
                        # Regular directory
                        item_count = len(list(item.iterdir()))
                        print(f'   📁 {item.name}/ ({item_count} items)')
                else:
                    # Regular file
                    size_mb = item.stat().st_size / 1024 / 1024
                    file_ext = item.suffix.lower()
                    if file_ext == '.parquet':
                        print(f'   📦 {item.name} (Parquet, {size_mb:.2f} MB)')
                    else:
                        print(f'   ? {item.name} ({size_mb:.2f} MB)')
    
    # Also check for files directly in bronze directory
    direct_files = [f for f in BRONZE_DIR.iterdir() if f.is_file()]
    if direct_files:
        print(f'\n📋 Files in Bronze Root ({len(direct_files)} files):')
        total_size = 0
        for f in sorted(direct_files):
            size_mb = f.stat().st_size / 1024 / 1024
            total_size += size_mb
            if f.suffix.lower() == '.parquet':
                print(f'   ? {f.name} (Parquet, {size_mb:.2f} MB)')
            else:
                print(f'   📄 {f.name} ({size_mb:.2f} MB)')
        
        print(f'   📊 Total root files size: {total_size:.2f} MB')
        
        # Try to show content of a parquet file if any exist
        parquet_files = [f for f in direct_files if f.suffix == '.parquet' and 'manifest' not in f.name]
        if parquet_files:
            print(f'\n🔍 Preview of {parquet_files[0].name}:')
            try:
                if spark is not None:
                    df_preview = spark.read.parquet(str(parquet_files[0]))
                    df_preview.show(5)
                else:
                    df_preview = pd.read_parquet(str(parquet_files[0]))
                    display(df_preview.head())
            except Exception as e:
                print(f'   Could not preview file: {e}')
        
    # Check downloads directory
    if DOWNLOAD_DIR.exists():
        download_files = list(DOWNLOAD_DIR.glob('*'))
        download_files = [f for f in download_files if f.is_file()]
        if download_files:
            print(f'\n📥 Files in Downloads Directory ({len(download_files)} files):')
            for f in sorted(download_files)[:5]:  # Show first 5
                size_mb = f.stat().st_size / 1024 / 1024
                print(f'   📁 {f.name} ({size_mb:.2f} MB)')
            if len(download_files) > 5:
                print(f'   ... and {len(download_files) - 5} more files')
        else:
            print(f'\n📥 Downloads Directory: Empty')
            
else:
    print('❌ Bronze directory not found - run the download cell first!')

print(f'\n🔺 Delta Lake Status:')
if spark is not None:
    try:
        from delta.tables import DeltaTable
        print('   ✅ Delta Lake libraries available')
        print('   ✅ Ready for Delta pipeline automation')
    except ImportError:
        print('   ⚠️  Delta Lake libraries not installed')
        print('   💡 Install with: pip install delta-spark')
else:
    print('   ⚠️  No Spark session available')
    print('   💡 Fabric environments should have Spark available')

📝 Creating ingestion manifest and verifying Fabric lakehouse structure...
🔍 Verifying Fabric lakehouse directory structure:
   Lakehouse root: lakehouse\default\Files
   Bronze base: lakehouse\default\Files\bronze\OMIE
   Bronze OMIE: lakehouse\default\Files\bronze\OMIE
   Downloads: lakehouse\default\Files\bronze\OMIE\downloads

📂 Contents of lakehouse\default\Files\bronze\OMIE:
   📁 2023/ (365 items)
   📁 2024/ (366 items)
   📁 2025/ (274 items)
   📁 downloads/ (0 items)
   📄 omie_ingestion_manifest.csv (0.24 MB)
   📄 omie_ingestion_manifest.parquet (0.05 MB)
✅ Manifest saved: lakehouse\default\Files\bronze\OMIE\omie_ingestion_manifest.parquet
✅ CSV manifest saved: lakehouse\default\Files\bronze\OMIE\omie_ingestion_manifest.csv

📊 Ingestion Summary:
   Files processed this session: 1005
   Total size: 5.10 MB
   Lakehouse location: lakehouse\default\Files\bronze\OMIE

📁 Current Lakehouse Structure:
   Lakehouse root: lakehouse\default\Files
   Bronze directory: lakehouse\default\File

: 

: 

In [ ]:
# Resume processing for missing years (2024 and 2025)
print("🔄 Resuming OMIE data processing for missing years...")

# Check which years are missing or incomplete
def check_missing_years():
    """Check which years need processing"""
    target_years = [2023, 2024, 2025]
    year_status = {}
    
    for year in target_years:
        year_dir = BRONZE_DIR / str(year)
        if year_dir.exists():
            files_count = len([f for f in year_dir.glob('*.parquet')])
            year_status[year] = {
                'exists': True,
                'files_count': files_count,
                'needs_processing': files_count < 300  # Rough estimate for incomplete year
            }
            print(f"📅 {year}: {files_count} files found")
        else:
            year_status[year] = {
                'exists': False,
                'files_count': 0,
                'needs_processing': True
            }
            print(f"❌ {year}: No directory found")
    
    return year_status

# Check current status
year_status = check_missing_years()

# Identify years that need processing
missing_years = [year for year, status in year_status.items() if status['needs_processing']]

if missing_years:
    print(f"\n🎯 Years needing processing: {missing_years}")
    print("💡 Solution: Re-run cell 8 (main processing cell) to continue with missing years")
    print("   The notebook will automatically skip existing files and process only missing ones")
    
    # If we have the files list from previous session, filter for missing years
    if 'files' in globals() and files:
        print(f"\n📋 Files available from previous session: {len(files)} total")
        
        # Quick re-filter for missing years only
        from datetime import datetime
        
        def extract_date_from_filename(filename):
            """Extract date from OMIE filename"""
            import re
            date_patterns = [r'(\d{8})', r'(\d{4}\d{2}\d{2})']
            for pattern in date_patterns:
                match = re.search(pattern, filename)
                if match:
                    date_str = match.group(1)
                    if len(date_str) == 8:
                        year = int(date_str[:4])
                        return year, date_str
            return None, None
        
        # Filter files for missing years only
        missing_files = []
        for file_info in files:
            filename = file_info.get('name', '')
            year, date_str = extract_date_from_filename(filename)
            if year and year in missing_years:
                file_info['extracted_year'] = year
                file_info['extracted_date'] = date_str
                missing_files.append(file_info)
        
        print(f"📊 Files for missing years:")
        missing_stats = {}
        for file_info in missing_files:
            year = file_info['extracted_year']
            missing_stats[year] = missing_stats.get(year, 0) + 1
        
        for year in missing_years:
            count = missing_stats.get(year, 0)
            print(f"   {year}: {count} files to process")
        
        # Make available for processing
        globals()['filtered_files_resume'] = missing_files
        print(f"\n✅ Prepared {len(missing_files)} files for resume processing")
        print("🚀 Ready to resume! Re-run cell 8 to continue processing")
        
    else:
        print("\n⚠️  No files list found from previous session")
        print("💡 Solution: Re-run cells 5-8 to restart complete processing")
        print("   Existing files will be skipped automatically")

else:
    print("\n✅ All years appear to be complete!")
    
    # Show final summary
    total_files = sum(status['files_count'] for status in year_status.values())
    print(f"📊 Total files processed: {total_files}")
    print(f"📅 Years: {list(year_status.keys())}")
    print(f"📁 Location: {BRONZE_DIR}")

print(f"\n💡 Current status summary:")
for year, status in year_status.items():
    status_icon = "✅" if not status['needs_processing'] else "⚠️"
    print(f"   {status_icon} {year}: {status['files_count']} files")

## Next Steps & Usage

### Fabric Integration
- ✅ Files are saved to lakehouse `lkh_OMIE` in `Files/bronze/OMIE/{year}/`
- ✅ Parquet format optimized for Fabric analytics and visualization
- ✅ Manifest tracking for data lineage and processing history
- ✅ Year-based organization (2023, 2024, 2025) for efficient querying

### Data Pipeline Integration
1. **Silver Layer**: Create transformations to clean and standardize the hourly price data
2. **Gold Layer**: Create aggregated views (daily, monthly averages) for business consumption
3. **Scheduling**: Set up Fabric pipeline to run this notebook on a schedule
4. **Data Factory**: Integrate with other energy market data sources

### Key Features
- **Direct Parquet Conversion**: No intermediate files - downloads directly to Parquet format
- **Duplicate Detection**: Advanced URL deduplication prevents downloading identical files
- **Year Organization**: Files automatically sorted by extraction year for optimal analytics
- **Medallion Architecture**: Bronze layer ready for Silver transformations

### Monitoring
- Check the manifest file for ingestion history and file sizes
- Monitor lakehouse storage consumption in Fabric admin center
- Set up alerts for failed downloads or processing errors

In [ ]:
# Create Fabric Tables from Bronze Layer Data
print("🔥 Creating Fabric/Spark tables from Bronze layer data...")

# Ensure required variables and imports are available for this cell
from pathlib import Path
import os
import pandas as pd
from pyspark.sql.functions import lit

# Re-establish directory paths in case this cell is run independently
if 'BRONZE_DIR' not in globals():
    print("🔧 Setting up directory paths...")
    
    # In Fabric, we need to use mssparkutils to access the lakehouse properly
    try:
        # Check if we're in a Fabric environment with mssparkutils
        lakehouse_list = mssparkutils.fs.ls("/")
        
        # In Fabric, use the proper mount path
        if any(item.name == "Files" for item in lakehouse_list):
            LAKEHOUSE_ROOT = Path("/Files")
            print(f'✅ Detected Fabric environment with Files mount')
        else:
            LAKEHOUSE_ROOT = Path("/lakehouse/default/Files") 
            print(f'✅ Using standard Fabric lakehouse path')
            
    except NameError:
        # mssparkutils not available - local development
        if os.path.exists('/lakehouse/default/Files'):
            LAKEHOUSE_ROOT = Path('/lakehouse/default/Files')
            print(f'✅ Detected Fabric environment - using lakehouse: {LAKEHOUSE_ROOT}')
        else:
            LAKEHOUSE_ROOT = Path('lakehouse/default/Files')
            print(f'💻 Local development mode - using: {LAKEHOUSE_ROOT}')
    
    # Set up OMIE bronze directory
    BRONZE_DIR = LAKEHOUSE_ROOT / 'bronze' / 'OMIE'
    
    print(f"📁 Bronze directory path: {BRONZE_DIR}")
    
    # Check if directory exists using different methods
    dir_exists = False
    
    try:
        # Method 1: Check with pathlib
        dir_exists = BRONZE_DIR.exists()
        if dir_exists:
            print(f"✅ Bronze directory found (pathlib): {BRONZE_DIR}")
        
    except Exception as e1:
        try:
            # Method 2: Check with mssparkutils
            mssparkutils.fs.ls(str(BRONZE_DIR))
            dir_exists = True
            print(f"✅ Bronze directory found (mssparkutils): {BRONZE_DIR}")
            
        except Exception as e2:
            print(f"❌ Bronze directory not accessible")
            print(f"   Pathlib error: {e1}")
            print(f"   mssparkutils error: {e2}")
    
    if not dir_exists:
        print(f"❌ Bronze directory not found: {BRONZE_DIR}")
        print("💡 Please run the data ingestion cells first (cells 1-7)")
        print("   This will create the required directory structure and download data")
        print("\n🔍 Available directories:")
        try:
            # Show what's available in the root
            for item in mssparkutils.fs.ls(str(LAKEHOUSE_ROOT)):
                print(f"   {'📁' if item.isDir else '📄'} {item.name}")
        except:
            print("   Cannot list root directory contents")

# Re-establish Delta availability flag
if 'DELTA_AVAILABLE' not in globals():
    print("🔧 Setting up Delta Lake configuration...")
    # Force disable Delta Lake due to OneLake connectivity issues
    DELTA_AVAILABLE = False
    print("⚠️  Delta Lake disabled due to OneLake connectivity issues")
    print("   💡 Using Parquet format for table creation")

# Check if we have Spark available
if 'spark' not in globals() or spark is None:
    print("❌ No Spark session available. Cannot create tables.")
    print("💡 Make sure you're running in a Fabric Lakehouse environment")
    print("💡 Try running the earlier cells to initialize Spark")
elif not dir_exists:
    print("❌ No bronze data directory found. Cannot create tables.")
    print("💡 Please run the data ingestion process first")
else:
    print(f"✅ Spark session available: {spark.version}")
    print(f"✅ Bronze directory accessible: {BRONZE_DIR}")
    
    # Get list of all years with data using mssparkutils for better Fabric compatibility
    try:
        # Try using mssparkutils first
        bronze_items = mssparkutils.fs.ls(str(BRONZE_DIR))
        bronze_years = [item for item in bronze_items if item.isDir and item.name.isdigit()]
        bronze_years.sort(key=lambda x: x.name)
        
        print(f"📅 Found {len(bronze_years)} years of data: {[y.name for y in bronze_years]}")
        
        if bronze_years:
            tables_created = []
            
            for year_item in bronze_years:
                year = year_item.name
                print(f"\n🗓️  Processing {year} data...")
                
                # Build year directory path
                year_path = f"{BRONZE_DIR}/{year}"
                
                # Get all Parquet files for this year using mssparkutils
                try:
                    year_files = mssparkutils.fs.ls(year_path)
                    parquet_files = [f for f in year_files if f.name.endswith(".parquet")]
                    
                    if not parquet_files:
                        print(f"   ⚠️  No Parquet files found in {year}")
                        continue
                        
                    print(f"   📦 Found {len(parquet_files)} Parquet files")
                    
                    # Create unified table for the year
                    table_name = f"omie_bronze_{year}"
                    
                    # Read all Parquet files for this year into a single DataFrame
                    print(f"   📥 Reading Parquet files...")
                    parquet_pattern = f"{year_path}/*.parquet"
                    year_df = spark.read.parquet(parquet_pattern)
                    
                    # Add year partition column if not already present
                    if "_extraction_year" not in year_df.columns:
                        year_df = year_df.withColumn("_extraction_year", lit(int(year)))
                    
                    # Show basic info about the data
                    row_count = year_df.count()
                    column_count = len(year_df.columns)
                    
                    print(f"   📊 Data loaded: {row_count:,} rows, {column_count} columns")
                    
                    # Show sample of the data
                    print(f"   🔍 Sample data preview:")
                    year_df.show(3, truncate=False)
                    
                    # Create/register the table
                    print(f"   🔧 Creating table: {table_name}")
                    
                    # Write as Delta table if available, otherwise as Parquet
                    table_path = f"{BRONZE_DIR}/{table_name}_table"
                    
                    if DELTA_AVAILABLE:
                        # Create Delta table
                        year_df.write.format("delta").mode("overwrite").option("path", table_path).saveAsTable(table_name)
                        print(f"   🔺 Delta table created: {table_name}")
                    else:
                        # Create Parquet table
                        year_df.write.mode("overwrite").option("path", table_path).saveAsTable(table_name)
                        print(f"   📦 Parquet table created: {table_name}")
                    
                    # Verify table creation
                    spark.sql(f"DESCRIBE TABLE {table_name}").show()
                    
                    tables_created.append({
                        'table_name': table_name,
                        'year': year,
                        'rows': row_count,
                        'columns': column_count,
                        'path': table_path,
                        'format': 'delta' if DELTA_AVAILABLE else 'parquet'
                    })
                    
                    print(f"   ✅ Table {table_name} successfully created and registered")
                    
                except Exception as e:
                    print(f"   ❌ Failed to create table for {year}: {e}")
                    import traceback
                    traceback.print_exc()
        else:
            print("⚠️  No year directories found in bronze layer")
            tables_created = []
        
    except Exception as e:
        print(f"❌ Error accessing bronze directory: {e}")
        print("💡 This might indicate the data ingestion hasn't been completed yet")
        tables_created = []
                print(f"   📦 Parquet table created: {table_name}")
            
            # Verify table creation
            spark.sql(f"DESCRIBE TABLE {table_name}").show()
            
            tables_created.append({
                'table_name': table_name,
                'year': year,
                'rows': row_count,
                'columns': column_count,
                'path': str(table_path),
                'format': 'delta' if DELTA_AVAILABLE else 'parquet'
            })
            
            print(f"   ✅ Table {table_name} successfully created and registered")
            
        except Exception as e:
            print(f"   ❌ Failed to create table for {year}: {e}")
            import traceback
            traceback.print_exc()
    
    # Create unified view across all years
    if tables_created and len(tables_created) > 1:
        print(f"\n🔄 Creating unified view across all years...")
        
        try:
            # Create UNION view of all year tables
            union_query = " UNION ALL ".join([f"SELECT * FROM {t['table_name']}" for t in tables_created])
            
            view_name = "omie_bronze_all_years"
            spark.sql(f"CREATE OR REPLACE VIEW {view_name} AS {union_query}")
            
            # Test the view
            unified_df = spark.sql(f"SELECT COUNT(*) as total_rows, MIN(_extraction_year) as min_year, MAX(_extraction_year) as max_year FROM {view_name}")
            unified_df.show()
            
            print(f"   ✅ Unified view created: {view_name}")
            
        except Exception as e:
            print(f"   ⚠️  Could not create unified view: {e}")
    
    # Summary of created tables
    if tables_created:
        print(f"\n📋 Summary of Created Tables:")
        tables_df = pd.DataFrame(tables_created)
        display(tables_df)
        
        total_rows = sum(t['rows'] for t in tables_created)
        print(f"\n📊 Overall Statistics:")
        print(f"   📁 Tables created: {len(tables_created)}")
        print(f"   📊 Total rows: {total_rows:,}")
        print(f"   📅 Years covered: {sorted(set(t['year'] for t in tables_created))}")
        print(f"   📦 Format: {tables_created[0]['format'].title()}")
        
        # Test queries
        print(f"\n🔍 Test Queries You Can Run:")
        print(f"   # Show all available tables")
        print(f"   spark.sql('SHOW TABLES').show()")
        print(f"   ")
        print(f"   # Query specific year")
        print(f"   spark.sql('SELECT * FROM omie_bronze_2023 LIMIT 10').show()")
        print(f"   ")
        print(f"   # Query all years")
        if len(tables_created) > 1:
            print(f"   spark.sql('SELECT _extraction_year, COUNT(*) FROM omie_bronze_all_years GROUP BY _extraction_year ORDER BY _extraction_year').show()")
        
        # Make tables info available globally
        globals()['bronze_tables'] = tables_created
        
        print(f"\n✅ Bronze layer tables ready for SQL queries and pipeline integration!")
        print(f"🏗️  Next step: Deploy automation pipelines")
        
    else:
        print(f"\n❌ No tables were created successfully")
        print(f"💡 Check that Parquet files exist in year directories under {BRONZE_DIR}")

In [ ]:
# Deploy OMIE Automation Pipelines to Fabric Data Factory
print("🚀 Deploying OMIE automation pipelines to Fabric Data Factory...")

# Pipeline configurations (these were created in the comprehensive infrastructure)
daily_pipeline_config = {
    "name": "omie_daily_incremental",
    "description": "Daily OMIE data ingestion with incremental updates",
    "schedule": "6:00 AM UTC daily",
    "activities": [
        {
            "name": "RunOMIENotebook",
            "type": "Notebook",
            "notebook": "omie.ipynb",
            "parameters": {
                "incremental_mode": True,
                "target_date": "yesterday"
            }
        },
        {
            "name": "ValidateIngestion", 
            "type": "Validation",
            "dependsOn": ["RunOMIENotebook"]
        }
    ]
}

monthly_pipeline_config = {
    "name": "omie_monthly_maintenance",
    "description": "Monthly OMIE data maintenance and optimization",
    "schedule": "2:00 AM UTC first Sunday of month",
    "activities": [
        {
            "name": "RunFullValidation",
            "type": "Notebook", 
            "notebook": "omie_monthly_pipeline.py",
            "parameters": {
                "full_validation": True,
                "optimize_tables": True
            }
        },
        {
            "name": "UpdateManifest",
            "type": "Notebook",
            "dependsOn": ["RunFullValidation"]
        }
    ]
}

# Create pipeline files in the workspace
pipelines_dir = Path(r"c:\Users\USUARIO\desktop\factoria-f5\biap4\energy-management-control\scripts\automation")
pipelines_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Creating pipeline files in: {pipelines_dir}")

# Daily pipeline JSON for Fabric Data Factory
daily_pipeline_json = {
    "name": "omie_daily_incremental",
    "properties": {
        "description": "Daily OMIE data ingestion with incremental updates and change detection",
        "activities": [
            {
                "name": "RunOMIEIngestion",
                "type": "SparkJob",
                "dependsOn": [],
                "policy": {
                    "timeout": "02:00:00",
                    "retry": 2,
                    "retryIntervalInSeconds": 300
                },
                "typeProperties": {
                    "sparkPool": {
                        "referenceName": "default",
                        "type": "SparkPoolReference"
                    },
                    "file": {
                        "value": "/notebooks/bronze/OMIE/omie.ipynb",
                        "type": "Expression"
                    },
                    "args": [
                        "--incremental",
                        "--target-date",
                        "@formatDateTime(subtractFromTime(utcnow(), 1, 'Day'), 'yyyy-MM-dd')"
                    ]
                }
            },
            {
                "name": "ValidateDataQuality",
                "type": "Script", 
                "dependsOn": [
                    {
                        "activity": "RunOMIEIngestion",
                        "dependencyConditions": ["Succeeded"]
                    }
                ],
                "typeProperties": {
                    "scripts": [
                        {
                            "type": "Query",
                            "text": "SELECT COUNT(*) as daily_records FROM omie_bronze_all_years WHERE DATE(_ingested_at) = CURRENT_DATE()"
                        }
                    ]
                }
            }
        ],
        "triggers": [
            {
                "name": "DailyScheduleTrigger",
                "type": "ScheduleTrigger",
                "typeProperties": {
                    "recurrence": {
                        "frequency": "Day",
                        "interval": 1,
                        "startTime": "2024-01-01T06:00:00Z",
                        "timeZone": "UTC"
                    }
                }
            }
        ],
        "annotations": ["OMIE", "Bronze", "Daily"],
        "folder": {
            "name": "energy-management/omie"
        }
    }
}

# Monthly maintenance pipeline JSON
monthly_pipeline_json = {
    "name": "omie_monthly_maintenance", 
    "properties": {
        "description": "Monthly OMIE data maintenance, optimization, and quality validation",
        "activities": [
            {
                "name": "DataQualityValidation",
                "type": "Script",
                "typeProperties": {
                    "scripts": [
                        {
                            "type": "Query", 
                            "text": """
                            SELECT 
                                _extraction_year,
                                COUNT(*) as total_files,
                                SUM(CASE WHEN _source_file IS NOT NULL THEN 1 ELSE 0 END) as valid_files,
                                MIN(_ingested_at) as oldest_ingestion,
                                MAX(_ingested_at) as latest_ingestion
                            FROM omie_bronze_all_years 
                            GROUP BY _extraction_year 
                            ORDER BY _extraction_year
                            """
                        }
                    ]
                }
            },
            {
                "name": "OptimizeTables",
                "type": "SparkJob",
                "dependsOn": [
                    {
                        "activity": "DataQualityValidation", 
                        "dependencyConditions": ["Succeeded"]
                    }
                ],
                "typeProperties": {
                    "sparkPool": {
                        "referenceName": "default",
                        "type": "SparkPoolReference"
                    },
                    "file": {
                        "value": "/scripts/automation/omie_monthly_pipeline.py",
                        "type": "Expression"
                    }
                }
            },
            {
                "name": "UpdateManifest",
                "type": "SparkJob",
                "dependsOn": [
                    {
                        "activity": "OptimizeTables",
                        "dependencyConditions": ["Succeeded"] 
                    }
                ],
                "typeProperties": {
                    "sparkPool": {
                        "referenceName": "default", 
                        "type": "SparkPoolReference"
                    },
                    "file": {
                        "value": "/notebooks/bronze/OMIE/omie.ipynb",
                        "type": "Expression"
                    },
                    "args": ["--update-manifest-only"]
                }
            }
        ],
        "triggers": [
            {
                "name": "MonthlyMaintenanceTrigger",
                "type": "ScheduleTrigger", 
                "typeProperties": {
                    "recurrence": {
                        "frequency": "Month",
                        "interval": 1,
                        "startTime": "2024-01-07T02:00:00Z",
                        "timeZone": "UTC",
                        "schedule": {
                            "weekDays": ["Sunday"],
                            "hours": [2],
                            "minutes": [0]
                        }
                    }
                }
            }
        ],
        "annotations": ["OMIE", "Bronze", "Maintenance", "Monthly"],
        "folder": {
            "name": "energy-management/omie"
        }
    }
}

# Save pipeline JSON files
import json

daily_pipeline_file = pipelines_dir / "omie_daily_pipeline.json"
monthly_pipeline_file = pipelines_dir / "omie_monthly_pipeline.json"

with open(daily_pipeline_file, 'w') as f:
    json.dump(daily_pipeline_json, f, indent=2)

with open(monthly_pipeline_file, 'w') as f:
    json.dump(monthly_pipeline_json, f, indent=2)

print(f"✅ Pipeline files created:")
print(f"   📅 Daily: {daily_pipeline_file}")
print(f"   🗓️  Monthly: {monthly_pipeline_file}")

# Create deployment instructions
deployment_instructions = f"""
# OMIE Pipeline Deployment Instructions

## Prerequisites
1. Ensure lakehouse 'lkh_OMIE' exists in your Fabric workspace
2. Upload this notebook to: /notebooks/bronze/OMIE/omie.ipynb
3. Upload pipeline scripts to: /scripts/automation/

## Pipeline Deployment Steps

### 1. Daily Incremental Pipeline
- **Name**: omie_daily_incremental
- **Schedule**: 6:00 AM UTC daily
- **Purpose**: Download latest OMIE data with incremental updates
- **JSON File**: {daily_pipeline_file}

### 2. Monthly Maintenance Pipeline  
- **Name**: omie_monthly_maintenance
- **Schedule**: 2:00 AM UTC first Sunday of month
- **Purpose**: Data validation, optimization, and maintenance
- **JSON File**: {monthly_pipeline_file}

## Manual Deployment in Fabric

1. **Open Fabric Data Factory**
   - Go to your Fabric workspace
   - Select "Data Factory" from the left menu

2. **Import Pipeline JSON**
   - Click "New" → "Pipeline"
   - Click "Code view" in the pipeline editor
   - Copy/paste content from the JSON files above
   - Save the pipeline

3. **Configure Triggers**
   - Go to "Triggers" tab in pipeline
   - Enable the schedule triggers
   - Adjust timezone if needed

4. **Test Pipelines**
   - Run daily pipeline manually first
   - Verify data ingestion works correctly
   - Enable automatic triggers

## Monitoring & Alerts

- Set up email notifications for pipeline failures
- Monitor lakehouse storage usage
- Check data quality metrics monthly
- Review execution logs in Fabric monitoring

## Files Structure
```
lakehouse/default/Files/
├── bronze/OMIE/
│   ├── 2023/           # Historical data
│   ├── 2024/           # Current year data  
│   ├── 2025/           # Future data
│   └── omie_ingestion_manifest.parquet
└── scripts/automation/
    ├── omie_daily_pipeline.py
    └── omie_monthly_pipeline.py
```

## Next Steps
1. ✅ Deploy these pipelines to Fabric Data Factory
2. 🔄 Test pipeline execution
3. 🥈 Create Silver layer transformations
4. 🥇 Build Gold layer aggregations
"""

# Save deployment instructions
instructions_file = pipelines_dir / "DEPLOYMENT_INSTRUCTIONS.md"
with open(instructions_file, 'w') as f:
    f.write(deployment_instructions)

print(f"\n📋 Deployment instructions saved: {instructions_file}")

print(f"\n🎯 Pipeline Deployment Summary:")
print(f"   📅 Daily Pipeline: Runs at 6:00 AM UTC daily")
print(f"   🗓️  Monthly Pipeline: Runs at 2:00 AM UTC first Sunday of month") 
print(f"   📁 Files location: {pipelines_dir}")
print(f"   📋 Instructions: {instructions_file}")

print(f"\n💡 Next Steps:")
print(f"   1. Import the JSON files into Fabric Data Factory")
print(f"   2. Configure and test the pipelines")
print(f"   3. Enable automatic triggers")
print(f"   4. Create Silver layer transformations")

# Make pipeline configs available
globals()['daily_pipeline_json'] = daily_pipeline_json
globals()['monthly_pipeline_json'] = monthly_pipeline_json
globals()['pipelines_dir'] = pipelines_dir

print(f"\n✅ Automation pipelines ready for deployment!")